## Retriever And Chain With Langchain


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Jenkins-Readme.pdf")
docs = loader.load()
docs

[Document(page_content='SimplifyQA Pipeline Connector is an update to our existing \nJenkins Plugin with improved features.  It offers seamless \nintegration with SimplifyQA tool and triggers automated \nscripts execution.  \nThis plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins \nversion –  \nNew features that are added to this improved utility are controlling the Build Failure status based on \nthe percentage of failed testcases, improved timeouts and support to different types of Jenkins \nproject.  \nFollow below steps for Integration  \n1. SimplifyQA Agent should be installed and registered in the Host system. Login to \nSimplifyQA application, follow the steps in Help Guide to Download and install \nthe Agent and register the machine in which installation is done.  \n2. Create Suite of automated testcases that needs to be executed as part of CICD \npipeline.  \n3. Create a Pipeline with required suite and provide details of registered m

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='SimplifyQA Pipeline Connector is an update to our existing \nJenkins Plugin with improved features.  It offers seamless \nintegration with SimplifyQA tool and triggers automated \nscripts execution.  \nThis plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins \nversion –  \nNew features that are added to this improved utility are controlling the Build Failure status based on \nthe percentage of failed testcases, improved timeouts and support to different types of Jenkins \nproject.  \nFollow below steps for Integration  \n1. SimplifyQA Agent should be installed and registered in the Host system. Login to \nSimplifyQA application, follow the steps in Help Guide to Download and install \nthe Agent and register the machine in which installation is done.  \n2. Create Suite of automated testcases that needs to be executed as part of CICD \npipeline.  \n3. Create a Pipeline with required suite and provide details of registered m

In [4]:
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='SimplifyQA Pipeline Connector is an update to our existing \nJenkins Plugin with improved features.  It offers seamless \nintegration with SimplifyQA tool and triggers automated \nscripts execution.  \nThis plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins \nversion –  \nNew features that are added to this improved utility are controlling the Build Failure status based on \nthe percentage of failed testcases, improved timeouts and support to different types of Jenkins \nproject.  \nFollow below steps for Integration  \n1. SimplifyQA Agent should be installed and registered in the Host system. Login to \nSimplifyQA application, follow the steps in Help Guide to Download and install \nthe Agent and register the machine in which installation is done.  \n2. Create Suite of automated testcases that needs to be executed as part of CICD \npipeline.  \n3. Create a Pipeline with required suite and provide details of registered m

In [ ]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())
db = FAISS.from_documents(
    documents[:30],
    OllamaEmbeddings(
        model="llama3",
        base_url="http://localhost:11434",
        show_progress=True,
        temperature=0.2,
    ),
)

In [5]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# db.save_local("faiss_index")
db = FAISS.load_local(
    "faiss_index",
    OllamaEmbeddings(
        model="llama3",
        base_url="http://localhost:11434",
        show_progress=True,
        temperature=0.9,
    ),
    allow_dangerous_deserialization=True,
)
db

In [6]:
query = "What is Simplifyqa pipeline connector?"
result = db.similarity_search(query)
result[0].page_content

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


'About SimplifyQA  \nWelcome to SimplifyQA , the only Application Management Tool (ALM) you’ll ever need. SimplifyQA , is \ndesigned to make application management and testing easy, fast and reliable for anyone, regardless of \ncoding skills and technical knowledge. With update on Jenkins Utility, we o ffer better support and cover \nwide variety of projects to give you a more awesome CICD integration.'

In [9]:
for res in result:
    print(res.page_content)

About SimplifyQA  
Welcome to SimplifyQA , the only Application Management Tool (ALM) you’ll ever need. SimplifyQA , is 
designed to make application management and testing easy, fast and reliable for anyone, regardless of 
coding skills and technical knowledge. With update on Jenkins Utility, we o ffer better support and cover 
wide variety of projects to give you a more awesome CICD integration.
10. Results would also be available in SimplifyQA reports section. There would also 
be an easy navigation to SimplifyQA reports section from logs. Users can view the 
report and download as PDF.  
11. Tool can also trigger email to mentioned users, post completion of execution.
SimplifyQA Pipeline Connector is an update to our existing 
Jenkins Plugin with improved features.  It offers seamless 
integration with SimplifyQA tool and triggers automated 
scripts execution.  
This plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins 
version –  
New featur

In [10]:
from langchain_community.llms import Ollama

## Load Ollama LAMA3 LLM model
llm = Ollama(
    model="llama3",
    base_url="http://localhost:11434",
    temperature=0.9,
)
llm

Ollama(model='llama3', temperature=0.9)

In [11]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}"""
)

In [12]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AECE9D7DF0>)

In [14]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "Tell me something about SimplifyQA pipeline connector"})

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


ChunkedEncodingError: Response ended prematurely

In [ ]:
response["answer"]